## Group 12 Coursework 

In [1]:
import os
import datetime
import time
import json
import http.client
import requests
from html.parser import HTMLParser
from html.entities import name2codepoint


class MyHTMLParser(HTMLParser):
# This is a class for parsing the html returned from the web service endpoint

    def __init__(self):
        super(MyHTMLParser, self).__init__()
        self.emotion_type = None
        self.emotion_return = False
        
    def handle_starttag(self, tag, attrs):
        if tag=='h3':
            self.emotion_return = True
    
    def handle_data(self, data):
        if self.emotion_return:
            self.emotion_type = data.strip()
        
    def handle_endtag(self, tag):
        if tag=='h3':
            self.emotion_return = False

def deploy_changes(commitmsg):
    '''Run git commands to deploy current changes in project folder to Github and deploy to web server'''
    
    !git add .
    !git commit -m commitmsg
    !git push origin master
    print('Changes were deployed to Github. Please action the CI/CD pipeline running status.')
    
    

Below cell will read user input text, and then make a HTTP request using POST method to the webservice endpoint deployed locally (http://127.0.0.1:5000) to classify the emotion of the user input text.

In [2]:
usertext = input("Please input your text:  ")

Please input your text:  hello


In [7]:
data = {'input_text': usertext}
dbipaddr = '127.0.0.1:5000'

response = requests.post('http://' + dbipaddr, data=data)

parser = MyHTMLParser()

if response.status_code==200:       
    parser.feed(response.text)
    if parser.emotion_type is not None:
        print(f'The classified emotion type is: {parser.emotion_type}')
    else:
        print(f'No emotion can be classified!')

The classified emotion type is: neutral


In [6]:
deploy_changes("updated index.py to run on all ports.")

[master 98d44ff] commitmsg
 7 files changed, 242 insertions(+), 153 deletions(-)
 create mode 100644 .ipynb_checkpoints/run_app-checkpoint.ipynb
 rename Part1.ipynb => IndividualCoursework/Part1.ipynb (100%)
 rename Part2.ipynb => IndividualCoursework/Part2.ipynb (100%)
 rename Part3.ipynb => IndividualCoursework/Part3.ipynb (100%)
 delete mode 100644 test.py
Changes were deployed to Github. Please action the CI/CD pipeline running status.


remote: Resolving deltas:   0% (0/1)        
remote: Resolving deltas: 100% (1/1)        
remote: Resolving deltas: 100% (1/1), completed with 1 local object.        
To https://github.com/wongp1984/comm061nlp2.git
   fb893df..98d44ff  master -> master
